# Baseline: Christmas Tree Packing

This notebook implements a baseline solution for the Santa 2025 Christmas Tree Packing challenge.

**Approach:**
1. Load and score the sample submission
2. Implement greedy placement with collision detection
3. Apply rotation optimization (fix_direction)
4. Generate improved submission

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import random
import time

# Set precision
getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# Tree geometry constants
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def load_submission(filepath):
    """Load a submission CSV and return dict of tree lists."""
    df = pd.read_csv(filepath)
    df['x'] = df['x'].str.strip('s').astype(float)
    df['y'] = df['y'].str.strip('s').astype(float)
    df['deg'] = df['deg'].str.strip('s').astype(float)
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    df['group_id'] = df['group_id'].astype(int)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg'])
                     for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
    
    return dict_of_tree_list

def get_side_length(trees):
    """Calculate the side length of the bounding square."""
    if not trees:
        return 0
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return float(max(width, height))

def calculate_total_score(dict_of_tree_list):
    """Calculate total score for all configurations."""
    total_score = 0
    for n, trees in dict_of_tree_list.items():
        side = get_side_length(trees)
        score = side ** 2 / n
        total_score += score
    return total_score

print("Utility functions defined")

Utility functions defined


In [4]:
# Load and score the sample submission
sample_path = '/home/data/sample_submission.csv'
dict_of_tree_list = load_submission(sample_path)

sample_score = calculate_total_score(dict_of_tree_list)
print(f"Sample submission score: {sample_score:.6f}")
print(f"Target score to beat: 68.922808")

Sample submission score: 173.652299
Target score to beat: 68.922808


In [5]:
# Rotation optimization (fix_direction)
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    if not trees:
        return 0, 0.0
    
    # Collect all points from tree polygons
    all_points = []
    for tree in trees:
        coords = list(tree.polygon.exterior.coords)
        for x, y in coords:
            all_points.append([float(Decimal(x) / scale_factor), float(Decimal(y) / scale_factor)])
    
    points_np = np.array(all_points)
    
    # Use convex hull for efficiency
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        hull_points = points_np
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    # Find optimal angle
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    
    if improvement > 1e-8:
        return found_side, found_angle_deg
    else:
        return initial_side, 0.0

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees."""
    if not trees or abs(angle_deg) < 1e-9:
        return trees
    
    # Get bounding box center
    all_points = []
    for tree in trees:
        coords = list(tree.polygon.exterior.coords)
        for x, y in coords:
            all_points.append([float(Decimal(x) / scale_factor), float(Decimal(y) / scale_factor)])
    points_np = np.array(all_points)
    
    min_x, min_y = points_np.min(axis=0)
    max_x, max_y = points_np.max(axis=0)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    # Rotate tree centers
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    # Create new trees with rotated positions and angles
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            center_x=str(rotated[i, 0]),
            center_y=str(rotated[i, 1]),
            angle=str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    
    return rotated_trees

print("Rotation optimization functions defined")

Rotation optimization functions defined


In [6]:
# Apply rotation optimization to all configurations
print("Applying rotation optimization to all configurations...")
start_time = time.time()

optimized_trees = {}
total_improvement = 0

for n in range(1, 201):
    trees = dict_of_tree_list[n]
    original_side = get_side_length(trees)
    
    # Find optimal rotation
    new_side, best_angle = optimize_rotation(trees)
    
    if best_angle > 0.001:
        # Apply rotation
        rotated_trees = apply_rotation(trees, best_angle)
        optimized_trees[n] = rotated_trees
        improvement = original_side - new_side
        total_improvement += improvement
        if n <= 10 or n % 50 == 0:
            print(f"N={n}: side {original_side:.6f} -> {new_side:.6f} (angle={best_angle:.2f}°, improvement={improvement:.6f})")
    else:
        optimized_trees[n] = trees

print(f"\nTotal time: {time.time() - start_time:.2f}s")
print(f"Total side length improvement: {total_improvement:.6f}")

Applying rotation optimization to all configurations...
N=1: side 1.000000 -> 0.813173 (angle=45.00°, improvement=0.186827)
N=2: side 1.211271 -> 1.190975 (angle=66.37°, improvement=0.020296)
N=3: side 1.670600 -> 1.430486 (angle=36.78°, improvement=0.240114)
N=4: side 2.039257 -> 1.792668 (angle=19.18°, improvement=0.246589)
N=5: side 2.121716 -> 2.031056 (angle=66.37°, improvement=0.090660)
N=6: side 2.172745 -> 2.079765 (angle=59.44°, improvement=0.092980)
N=7: side 2.901647 -> 2.468822 (angle=48.21°, improvement=0.432825)
N=8: side 3.441115 -> 2.589799 (angle=55.94°, improvement=0.851316)
N=9: side 3.441115 -> 2.615259 (angle=57.82°, improvement=0.825856)
N=10: side 3.441115 -> 2.615259 (angle=57.82°, improvement=0.825856)


N=50: side 6.649242 -> 6.195115 (angle=59.21°, improvement=0.454127)


N=100: side 8.956058 -> 8.673279 (angle=49.37°, improvement=0.282779)


N=150: side 11.098935 -> 10.614321 (angle=49.38°, improvement=0.484614)


N=200: side 13.034470 -> 12.382298 (angle=21.92°, improvement=0.652172)

Total time: 5.10s
Total side length improvement: 101.184579


In [7]:
# Calculate new score
optimized_score = calculate_total_score(optimized_trees)
print(f"\nOriginal sample score: {sample_score:.6f}")
print(f"Optimized score: {optimized_score:.6f}")
print(f"Score improvement: {sample_score - optimized_score:.6f}")
print(f"Target score: 68.922808")


Original sample score: 173.652299
Optimized score: 151.174315
Score improvement: 22.477984
Target score: 68.922808


In [8]:
# Save submission
def save_submission(dict_of_tree_list, filepath):
    """Save trees to submission CSV format."""
    rows = []
    for n in range(1, 201):
        trees = dict_of_tree_list[n]
        for i, tree in enumerate(trees):
            row = {
                'id': f'{n:03d}_{i}',
                'x': f's{float(tree.center_x):.6f}',
                'y': f's{float(tree.center_y):.6f}',
                'deg': f's{float(tree.angle):.6f}'
            }
            rows.append(row)
    
    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)
    print(f"Saved submission to {filepath}")
    return df

# Save optimized submission
submission_df = save_submission(optimized_trees, '/home/submission/submission.csv')
print(f"\nSubmission shape: {submission_df.shape}")
print(submission_df.head(10))

Saved submission to /home/submission/submission.csv

Submission shape: (20100, 4)
      id           x           y          deg
0  001_0  s-0.087868   s0.212132  s135.000000
1  002_0  s-0.353219   s0.028806  s156.370623
2  002_1   s0.196446   s0.009617  s156.370623
3  003_0  s-0.137830  s-0.069298  s126.781219
4  003_1   s0.330676  s-0.357398  s126.781219
5  003_2   s0.172905   s0.384512  s216.781219
6  004_0  s-0.115248   s0.030885  s109.178445
7  004_1   s0.244195  s-0.385410  s109.178445
8  004_2   s0.318177   s0.369474  s199.178445
9  004_3  s-0.813341  s-0.454055  s199.178445


In [9]:
# Verify submission by reloading and scoring
print("\nVerifying submission...")
verify_trees = load_submission('/home/submission/submission.csv')
verify_score = calculate_total_score(verify_trees)
print(f"Verified score: {verify_score:.6f}")


Verifying submission...


Verified score: 151.174322


In [10]:
# Check for overlaps in a few configurations
def has_overlap(trees):
    """Check if any two trees overlap."""
    if len(trees) <= 1:
        return False
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Checking for overlaps...")
overlap_count = 0
for n in range(1, 201):
    if has_overlap(optimized_trees[n]):
        overlap_count += 1
        if overlap_count <= 5:
            print(f"  Overlap detected in N={n}")

if overlap_count == 0:
    print("No overlaps detected!")
else:
    print(f"Total configurations with overlaps: {overlap_count}")

Checking for overlaps...


No overlaps detected!


In [11]:
# Final summary
print("="*50)
print("BASELINE EXPERIMENT SUMMARY")
print("="*50)
print(f"Original sample score: {sample_score:.6f}")
print(f"Optimized score (with rotation): {optimized_score:.6f}")
print(f"Score improvement: {sample_score - optimized_score:.6f}")
print(f"Target score: 68.922808")
print(f"Gap to target: {optimized_score - 68.922808:.6f}")
print("="*50)

BASELINE EXPERIMENT SUMMARY
Original sample score: 173.652299
Optimized score (with rotation): 151.174315
Score improvement: 22.477984
Target score: 68.922808
Gap to target: 82.251507
